In [ ]:
%matplotlib inline
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display


# Pitch and Autocorrelation

The [autocorrelation](http://en.wikipedia.org/wiki/Autocorrelation) of a signal describes the similarity of a signal against a time-shifted version of itself. 

For a signal $x$, the autocorrelation $r$ is:

$$ r(k) = \sum_n x(n) x(n-k) $$

In this equation, $k$ is often called the **lag** parameter. $r(k)$ is maximized at $k = 0$ and is symmetric about $k$.

The autocorrelation is useful for finding repeated patterns in a signal. For example, at short lags, the autocorrelation can tell us something about the signal's fundamental frequency. For longer lags, the autocorrelation may tell us something about the tempo of a musical signal.

Let's analyse some signals:

- Load the file 'sinusoid.wav'

In [ ]:
x, sr = librosa.load('sinusoid.wav')
ipd.Audio(x, rate=sr)

Plot the waveform

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr)

There are several ways we can compute the autocorrelation in Python. Here we will use [`librosa.autocorrelate`](http://bmcfee.github.io/librosa/generated/librosa.core.autocorrelate.html#librosa.core.autocorrelate):

In [ ]:
r = librosa.autocorrelate(x, max_size=10000)

Let's plot the result of the autocorrelation function:

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(r)
plt.xlabel('Lag (samples)')
plt.xlim(0, 10000)

`librosa.autocorrelate` conveniently only keeps one half of the autocorrelation function, since the autocorrelation is symmetric. Also, the `max_size` parameter prevents unnecessary calculations.

## Pitch Estimation

The autocorrelation is used to find repeated patterns within a signal. For musical signals, a repeated pattern can correspond to a pitch period. We can therefore use the autocorrelation function to estimate the pitch in a musical signal.

In [ ]:
x, sr = librosa.load('oboe_c6.wav')
ipd.Audio(x, rate=sr)

Compute and plot the autocorrelation:

In [ ]:
r = librosa.autocorrelate(x, max_size=5000)
plt.figure(figsize=(14, 5))
plt.plot(r[:200])

The autocorrelation always has a maximum at zero, i.e. zero lag. We want to identify the maximum outside of the peak centered at zero. Therefore, we might choose only to search within a range of reasonable pitches:

In [ ]:
midi_hi = 120.0
midi_lo = 12.0
f_hi = librosa.midi_to_hz(midi_hi)
f_lo = librosa.midi_to_hz(midi_lo)
t_lo = sr/f_hi
t_hi = sr/f_lo

In [ ]:
print(f_lo, f_hi)
print(t_lo, t_hi)

Set invalid pitch candidates to zero:

In [ ]:
r[:int(t_lo)] = 0
r[int(t_hi):] = 0

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(r[:1400])

Find the location of the maximum:

In [ ]:
t_max = r.argmax()
print(t_max)

Finally, estimate the pitch in Hertz:

In [ ]:
float(sr)/t_max

Indeed, that is very close to the true frequency of C6:

In [ ]:
librosa.midi_to_hz(84)

## Try the **pitch estimation** with other sounds


Use other sources: sinusoid.wav, violin_FMP.wav, etc, in all the steps from computing autocorrelation to print the detection of f0/pitch in Hz.

Explore this with different monophonic sound files (e.g. download from freesound):
- Was it more difficult or easier than the original file? Why?
- What was the easier example?
- What was the more difficult?

*Note: for some of these examples the analysis windows may have to be adjusted* 


## Other uses of autocorrelation

As you've seen, the ACF may be used to estimate the periodicity of a signal, giving an estimate of it's fundamental frequency (or as our brain perceives it, the pitch).

To think until **next week**:
- What other main characteristics of the music signals may be characterised by periodicities?
- If so, does it make sense to use the autocorrelation function for those potential tasks?

